# Практическое занятие 8

## Вариант 11

In [132]:
class AHPCriteriaMatrix1:
	def __init__(self, a, b, c, d, e, f, g, h, i, j, k, l, m, n, o):
		self.matrix = pd.DataFrame({
			'Инфраструктура': 			[1, 1 / a, 1 / b, 1 / c, 1 / d, 1 / e],
			'Расположение':   			[a, 1, 1 / f, 1 / g, 1 / h, 1 / i],
			'Цена':           			[b, f, 1, 1 / j, 1 / k, 1 / l],
			'Площадь': 		  			[c, g, j, 1, 1 / m, 1 / n],
			'Качество ремонта': 		[d, h, k, m, 1, 1 / o],
			'Близость к общ. трансп.':  [e, i, l, n, o, 1]
		}, index=['Инфраструктура', 'Расположение', 'Цена', 'Площадь', 'Качество ремонта', 'Близость к общ. трансп.'])

	def get_initial_matrix(self):
		return self.matrix

	def add_priority_vector(self):
		self.matrix['Оценка вектора приоритета'] = self.matrix.sum(axis=1) / len(self.matrix.columns)

	def add_normalized_priority_vector(self):
		total_priority = self.matrix['Оценка вектора приоритета'].sum()
		self.matrix['Нормализованные оценки вектора приоритета'] = self.matrix['Оценка вектора приоритета'] / total_priority

	def add_column_sum(self):
		self.matrix['Сумма по столбцам'] = self.matrix.iloc[:, :6].sum(axis=0)

	def multiply_by_normalized_priority(self):
		self.matrix['Умножение на нормализованные оценки'] = self.matrix['Сумма по столбцам'] * self.matrix['Нормализованные оценки вектора приоритета']

	def calculate_L_max(self):
		return self.matrix["Умножение на нормализованные оценки"].sum()

	def calculate_consistency_index(self):
		return (self.calculate_L_max() - len(self.matrix.index)) / (len(self.matrix.index) - 1)

	def calculate_consistency_ratio(self):
		return self.calculate_consistency_index() / 1.24

# Пример использования класса
a, b, c, d, e, f, g, h, i, j, k, l, m, n, o = 2, 3, 3, 4, 5, 3, 4, 5, 6, 2, 2, 7, 1, 8, 4
ahp_matrix = AHPCriteriaMatrix1(a, b, c, d, e, f, g, h, i, j, k, l, m, n, o)

# Добавление оценок вектора приоритета и нормализованных оценок
ahp_matrix.add_priority_vector()
ahp_matrix.add_normalized_priority_vector()
ahp_matrix.add_column_sum()
ahp_matrix.multiply_by_normalized_priority()
L_max = ahp_matrix.calculate_L_max()
ahp_matrix.calculate_consistency_index()
ahp_matrix.calculate_consistency_ratio()

print(f"Отношение согласованности: {L_max}")
ahp_matrix = ahp_matrix.get_initial_matrix()
ahp_matrix

Отношение согласованности: 7.149619507057804


,Инфраструктура,Расположение,Цена,Площадь,Качество ремонта,Близость к общ. трансп.,Оценка вектора приоритета,Нормализованные оценки вектора приоритета,Сумма по столбцам,Умножение на нормализованные оценки
Инфраструктура,1.000000,2.000000,3.000000,3.000,4.00,5,3.000000,0.256833,2.616667,0.672046
Расположение,0.500000,1.000000,3.000000,4.000,5.00,6,3.250000,0.278235,3.950000,1.099030
Цена,0.333333,0.333333,1.000000,2.000,2.00,7,2.111111,0.180734,8.142857,1.471692
Площадь,0.333333,0.250000,0.500000,1.000,1.00,8,1.847222,0.158142,11.125000,1.759334
Качество ремонта,0.250000,0.200000,0.500000,1.000,1.00,4,1.158333,0.099166,13.250000,1.313949
Близость к общ. трансп.,0.200000,0.166667,0.142857,0.125,0.25,1,0.314087,0.026889,31.000000,0.833568


In [129]:
import pandas as pd
import numpy as np

class AHPCriteriaMatrix:
	"""

	"""
	def __init__(self, apartments_names, criteria_comparisons):
		self.apartments_names = apartments_names
		self.criteria_comparisons = criteria_comparisons
		self.matrices = self._create_matrices()

	def get_matrix(self, criterion):
		return self.matrices.get(criterion, None)

	def _create_matrices(self):
		matrices = {}
		for criterion, comparison_matrix in self.criteria_comparisons.items():
			matrix_data = {}
			for i, apartment_row in enumerate(self.apartments_names):
				row_data = []
				for j, apartment_col in enumerate(self.apartments_names):
					row_data.append(comparison_matrix[i][j])
				matrix_data[apartment_row] = row_data
			matrices[criterion] = pd.DataFrame(matrix_data, index=self.apartments_names, columns=self.apartments_names)
		return matrices

	def add_priority_vector(self):
		"""

		:return:
		"""
		for matrix in self.matrices.values():
			matrix['Оценка вектора приоритета'] = matrix.apply(lambda row: np.prod(row) ** (1 / len(row)), axis=1)

	def normalize_priority_vector(self):
		"""

		:return:
		"""
		for matrix in self.matrices.values():
			total_priority = matrix['Оценка вектора приоритета'].sum()
			matrix['Нормализованные оценки вектора приоритета'] = matrix['Оценка вектора приоритета'] / total_priority

	def calculate_column_sums(self):
		"""

		:return:
		"""
		for matrix in self.matrices.values():
			matrix['Сумма по столбцам'] = matrix.sum(axis=0)

	def calculate_L_max(self):
		"""

		:return:
		"""
		L_max_values = {}
		for criterion, matrix in self.matrices.items():
			matrix['Умножение на нормализованные оценки'] = matrix['Сумма по столбцам'] * matrix['Нормализованные оценки вектора приоритета']
			L_max = matrix['Умножение на нормализованные оценки'].sum()
			L_max_values[criterion] = L_max
		return L_max_values

	def calculate_consistency_index(self, L_max_values):
		"""

		:param L_max_values:
		:return:
		"""
		CI_values = {}
		for criterion, L_max in L_max_values.items():
			n = len(self.matrices[criterion].columns) - 4
			CI = (L_max - n) / (n - 1)
			CI_values[criterion] = CI
		return CI_values

	def calculate_consistency_ratio(self, CI_values):
		"""

		:param CI_values:
		:return:
		"""
		RI_values = {6: 1.24, 5: 1.12, 4: 0.90, 3: 0.58, 2: 0.00}
		CR_values = {}
		for criterion, CI in CI_values.items():
			n = len(self.matrices[criterion].columns) - 4
			RI = RI_values[n]
			CR = CI / RI * 100
			CR_values[criterion] = CR
		return CR_values



In [130]:

# Значения для парных сравнений по каждому критерию
# Инфраструктура
a0, b0, c0, d0, e0, f0, g0, h0, i0, j0, k0, l0, m0, n0, o0 = 2, 3, 3, 4, 5, 6, 6, 6, 5, 3, 4, 1, 2, 1, 1


# Расположение
a1, b1, c1, d1, e1, f1, g1, h1, i1, j1, k1, l1, m1, n1, o1 = 2, 3, 6, 6, 6, 5, 7, 6, 7, 4, 6, 7, 2, 2, 1


# Цена
a2, b2, c2, d2, e2, f2, g2, h2, i2, j2, k2, l2, m2, n2, o2 = 1, 2, 3, 6, 4, 4, 6, 6, 2, 2, 4, 4, 5, 3, 2


# Площадь
a3, b3, c3, d3, e3, f3, g3, h3, i3, j3, k3, l3, m3, n3, o3 = 1, 2, 4, 5, 5, 5, 9, 9, 9, 7, 6, 4, 3, 4, 2


# Качество ремонта
a4, b4, c4, d4, e4, f4, g4, h4, i4, j4, k4, l4, m4, n4, o4 = 1, 2, 3, 4, 5, 6, 6, 6, 7, 7, 5, 4, 2, 1, 1


# Близость к общественному транспорту
a5, b5, c5, d5, e5, f5, g5, h5, i5, j5, k5, l5, m5, n5, o5 = 1, 2, 3, 4, 6, 9, 7, 7, 7, 3, 2, 3, 3, 2, 2


# Создание словаря для парных сравнений каждого критерия
criteria_comparisons = {
	'Инфраструктура': [
		[1, 1/a0, 1/b0, 1/c0, 1/d0, 1/e0],
		[a0, 1, 1/f0, 1/g0, 1/h0, 1/i0],
		[b0, f0, 1, 1/j0, 1/k0, 1/l0],
		[c0, g0, j0, 1, 1/m0, 1/n0],
		[d0, h0, k0, m0, 1, 1/o0],
		[e0, i0, l0, n0, o0, 1]
	],
	'Расположение': [
		[1, 1/a1, 1/b1, 1/c1, 1/d1, 1/e1],
		[a1, 1, 1/f1, 1/g1, 1/h1, 1/i1],
		[b1, f1, 1, 1/j1, 1/k1, 1/l1],
		[c1, g1, j1, 1, 1/m1, 1/n1],
		[d1, h1, k1, m1, 1, 1/o1],
		[e1, i1, l1, n1, o1, 1]
	],
	'Цена': [
		[1, 1/a2, 1/b2, 1/c2, 1/d2, 1/e2],
		[a2, 1, 1/f2, 1/g2, 1/h2, 1/i2],
		[b2, f2, 1, 1/j2, 1/k2, 1/l2],
		[c2, g2, j2, 1, 1/m2, 1/n2],
		[d2, h2, k2, m2, 1, 1/o2],
		[e2, i2, l2, n2, o2, 1]
	],
	'Площадь': [
		[1, 1/a3, 1/b3, 1/c3, 1/d3, 1/e3],
		[a3, 1, 1/f3, 1/g3, 1/h3, 1/i3],
		[b3, f3, 1, 1/j3, 1/k3, 1/l3],
		[c3, g3, j3, 1, 1/m3, 1/n3],
		[d3, h3, k3, m3, 1, 1/o3],
		[e3, i3, l3, n3, o3, 1]
	],
	'Качество ремонта': [
		[1, 1/a4, 1/b4, 1/c4, 1/d4, 1/e4],
		[a4, 1, 1/f4, 1/g4, 1/h4, 1/i4],
		[b4, f4, 1, 1/j4, 1/k4, 1/l4],
		[c4, g4, j4, 1, 1/m4, 1/n4],
		[d4, h4, k4, m4, 1, 1/o4],
		[e4, i4, l4, n4, o4, 1]
	],
	'Близость к общественному транспорту': [
		[1, 1/a5, 1/b5, 1/c5, 1/d5, 1/e5],
		[a5, 1, 1/f5, 1/g5, 1/h5, 1/i5],
		[b5, f5, 1, 1/j5, 1/k5, 1/l5],
		[c5, g5, j5, 1, 1/m5, 1/n5],
		[d5, h5, k5, m5, 1, 1/o5],
		[e5, i5, l5, n5, o5, 1]
	]
}


# Имена квартир (альтернатив)
apartments_names = ['Квартира1', 'Квартира2', 'Квартира3', 'Квартира4', 'Квартира5', 'Квартира6']

# Создание объекта класса
ahp_matrices = AHPCriteriaMatrix(apartments_names, criteria_comparisons)
ahp_matrices.add_priority_vector()
ahp_matrices.normalize_priority_vector()
ahp_matrices.calculate_column_sums()
L_max_values = ahp_matrices.calculate_L_max()
CI_values = ahp_matrices.calculate_consistency_index(L_max_values)
CR_values = ahp_matrices.calculate_consistency_ratio(CI_values)

### Инфраструктура

In [131]:
infra_matrix = ahp_matrices.get_matrix('Инфраструктура')
print(f"Отношение согласованности: {CR_values['Инфраструктура']}")
infra_matrix

Отношение согласованности: 8.45846882093425


,Квартира1,Квартира2,Квартира3,Квартира4,Квартира5,Квартира6,Оценка вектора приоритета,Нормализованные оценки вектора приоритета,Сумма по столбцам,Умножение на нормализованные оценки
Квартира1,1.000000,2.000000,3.000000,3.0,4.0,5,2.667168,0.331987,2.616667,0.868699
Квартира2,0.500000,1.000000,6.000000,6.0,6.0,5,2.853639,0.355197,3.700000,1.314229
Квартира3,0.333333,0.166667,1.000000,3.0,4.0,1,0.934655,0.116338,11.583333,1.347583
Квартира4,0.333333,0.166667,0.333333,1.0,2.0,1,0.577350,0.071864,14.500000,1.042024
Квартира5,0.250000,0.166667,0.250000,0.5,1.0,1,0.416342,0.051823,18.000000,0.932809
Квартира6,0.200000,0.200000,1.000000,1.0,1.0,1,0.584804,0.072791,14.000000,1.019081


### Расположение

In [87]:
location_matrix = ahp_matrices.get_matrix('Расположение')
print(f"Отношение согласованности: {CR_values['Расположение']}")
location_matrix

Отношение согласованности: 9.226732998015704


,Квартира1,Квартира2,Квартира3,Квартира4,Квартира5,Квартира6,Оценка вектора приоритета,Нормализованные оценки вектора приоритета,Сумма по столбцам,Умножение на нормализованные оценки
Квартира1,1.000000,2.000000,3.000000,6.0,6.0,6,3.301927,0.364960,2.333333,0.851572
Квартира2,0.500000,1.000000,5.000000,7.0,6.0,7,3.004101,0.332041,3.652381,1.212741
Квартира3,0.333333,0.200000,1.000000,4.0,6.0,7,1.495787,0.165328,9.559524,1.580459
Квартира4,0.166667,0.142857,0.250000,1.0,2.0,2,0.536363,0.059284,19.000000,1.126392
Квартира5,0.166667,0.166667,0.166667,0.5,1.0,1,0.363708,0.040200,22.000000,0.884408
Квартира6,0.166667,0.142857,0.142857,0.5,1.0,1,0.345491,0.038187,24.000000,0.916486


### Цена

In [88]:
price_matrix = ahp_matrices.get_matrix('Цена')
print(f"Отношение согласованности: {CR_values['Цена']}")
price_matrix

Отношение согласованности: 9.80182146231647


,Квартира1,Квартира2,Квартира3,Квартира4,Квартира5,Квартира6,Оценка вектора приоритета,Нормализованные оценки вектора приоритета,Сумма по столбцам,Умножение на нормализованные оценки
Квартира1,1.000000,1.000000,2.00,3.000000,6.0,4,2.289428,0.294473,3.250000,0.957039
Квартира2,1.000000,1.000000,4.00,6.000000,6.0,2,2.569797,0.330535,3.083333,1.019150
Квартира3,0.500000,0.250000,1.00,2.000000,4.0,4,1.259921,0.162055,8.000000,1.296440
Квартира4,0.333333,0.166667,0.50,1.000000,5.0,3,0.864234,0.111161,12.533333,1.393212
Квартира5,0.166667,0.166667,0.25,0.200000,1.0,2,0.374929,0.048225,22.500000,1.085054
Квартира6,0.250000,0.500000,0.25,0.333333,0.5,1,0.416342,0.053551,16.000000,0.856819


### Площадь

In [89]:
area_matrix = ahp_matrices.get_matrix('Площадь')
print(f"Отношение согласованности: {CR_values['Площадь']}")
area_matrix

Отношение согласованности: 9.530996736378897


,Квартира1,Квартира2,Квартира3,Квартира4,Квартира5,Квартира6,Оценка вектора приоритета,Нормализованные оценки вектора приоритета,Сумма по столбцам,Умножение на нормализованные оценки
Квартира1,1.00,1.000000,2.000000,4.000000,5.0,5,2.418271,0.262602,3.150000,0.827196
Квартира2,1.00,1.000000,5.000000,9.000000,9.0,9,3.922981,0.426000,2.533333,1.079199
Квартира3,0.50,0.200000,1.000000,7.000000,6.0,4,1.600362,0.173785,8.559524,1.487513
Квартира4,0.25,0.111111,0.142857,1.000000,3.0,4,0.602047,0.065377,21.583333,1.411047
Квартира5,0.20,0.111111,0.166667,0.333333,1.0,2,0.367641,0.039922,24.500000,0.978100
Квартира6,0.20,0.111111,0.250000,0.250000,0.5,1,0.297582,0.032315,25.000000,0.807866


### Качество ремонта

In [90]:
repair_quality_matrix = ahp_matrices.get_matrix('Качество ремонта')
print(f"Отношение согласованности: {CR_values['Качество ремонта']}")
repair_quality_matrix

Отношение согласованности: 9.899588327842066


,Квартира1,Квартира2,Квартира3,Квартира4,Квартира5,Квартира6,Оценка вектора приоритета,Нормализованные оценки вектора приоритета,Сумма по столбцам,Умножение на нормализованные оценки
Квартира1,1.000000,1.000000,2.000000,3.0,4.0,5,2.220906,0.262641,3.283333,0.862337
Квартира2,1.000000,1.000000,6.000000,6.0,6.0,7,3.387859,0.400642,2.642857,1.058841
Квартира3,0.500000,0.166667,1.000000,7.0,5.0,4,1.505998,0.178097,9.592857,1.708457
Квартира4,0.333333,0.166667,0.142857,1.0,2.0,1,0.501314,0.059285,18.500000,1.096764
Квартира5,0.250000,0.166667,0.200000,0.5,1.0,1,0.401142,0.047438,19.000000,0.901329
Квартира6,0.200000,0.142857,0.250000,1.0,1.0,1,0.438846,0.051897,19.000000,0.986046


### Близость к общественному транспорту

In [91]:
nearest_transport_matrix = ahp_matrices.get_matrix('Близость к общественному транспорту')
print(f"Отношение согласованности: {CR_values['Близость к общественному транспорту']}")
nearest_transport_matrix

Отношение согласованности: 8.33014734715599


,Квартира1,Квартира2,Квартира3,Квартира4,Квартира5,Квартира6,Оценка вектора приоритета,Нормализованные оценки вектора приоритета,Сумма по столбцам,Умножение на нормализованные оценки
Квартира1,1.000000,1.000000,2.000000,3.000000,4.0,6,2.289428,0.265800,3.250000,0.863849
Квартира2,1.000000,1.000000,9.000000,7.000000,7.0,7,3.815834,0.443013,2.539683,1.125113
Квартира3,0.500000,0.111111,1.000000,3.000000,2.0,3,1.000000,0.116099,13.166667,1.528632
Квартира4,0.333333,0.142857,0.333333,1.000000,3.0,2,0.675774,0.078457,14.833333,1.163772
Квартира5,0.250000,0.142857,0.500000,0.333333,1.0,2,0.477845,0.055477,17.500000,0.970850
Квартира6,0.166667,0.142857,0.333333,0.500000,0.5,1,0.354483,0.041155,21.000000,0.864254


### Глобальные вектора приоритетов

In [125]:
# Предполагаем, что столбец с весами называется 'weights'
weights = ahp_matrix.get_initial_matrix()['Нормализованные оценки вектора приоритета']

# Расчет глобального вектора приоритетов для каждой квартиры
global_priorities = []
for df in [infra_matrix, location_matrix, price_matrix, area_matrix, repair_quality_matrix, nearest_transport_matrix]:
	global_priority = np.dot(weights, df['Нормализованные оценки вектора приоритета'])
	global_priorities.append(global_priority)


kv1 = [x['Нормализованные оценки вектора приоритета']['Квартира1']
		 for x in (infra_matrix, location_matrix, price_matrix, area_matrix, repair_quality_matrix, nearest_transport_matrix)]

kv2 = [x['Нормализованные оценки вектора приоритета']['Квартира2']
	   for x in (infra_matrix, location_matrix, price_matrix, area_matrix, repair_quality_matrix, nearest_transport_matrix)]

kv3 = [x['Нормализованные оценки вектора приоритета']['Квартира3']
	   for x in (infra_matrix, location_matrix, price_matrix, area_matrix, repair_quality_matrix, nearest_transport_matrix)]

kv4 = [x['Нормализованные оценки вектора приоритета']['Квартира4']
	   for x in (infra_matrix, location_matrix, price_matrix, area_matrix, repair_quality_matrix, nearest_transport_matrix)]

kv5 = [x['Нормализованные оценки вектора приоритета']['Квартира5']
	   for x in (infra_matrix, location_matrix, price_matrix, area_matrix, repair_quality_matrix, nearest_transport_matrix)]

kv6 = [x['Нормализованные оценки вектора приоритета']['Квартира6']
	   for x in (infra_matrix, location_matrix, price_matrix, area_matrix, repair_quality_matrix, nearest_transport_matrix)]


apartments = ["Квартира1", "Квартира2", "Квартира3", "Квартира4", "Квартира5", "Квартира6"]
#criteria = ["Инфраструктура", "Расположение", "Цена", "Площадь", "Качество ремонта", "Близость к общ. трансп."]

data = {
	"Квартира": apartments,
	"Инфраструктура": [kv1[0], kv2[0], kv3[0], kv4[0], kv5[0], kv6[0]],
	"Расположение": [kv1[1], kv2[1], kv3[1], kv4[1], kv5[1], kv6[1]],
	"Цена": [kv1[2], kv2[2], kv3[2], kv4[2], kv5[2], kv6[2]],
	"Площадь": [kv1[3], kv2[3], kv3[3], kv4[3], kv5[3], kv6[3]],
	"Качество ремонта": [kv1[4], kv2[4], kv3[4], kv4[4], kv5[4], kv6[4]],
	"Близость к общ. трансп.": [kv1[5], kv2[5], kv3[5], kv4[5], kv5[5], kv6[5]],
	"Глобальный вектор приоритетов": global_priorities
}





df = pd.DataFrame(data)
df


,Квартира,Инфраструктура,Расположение,Цена,Площадь,Качество ремонта,Близость к общ. трансп.,Глобальный вектор приоритетов
0,Квартира1,0.331987,0.364960,0.294473,0.262602,0.262641,0.265800,0.223581
1,Квартира2,0.355197,0.332041,0.330535,0.426000,0.400642,0.443013,0.230388
2,Квартира3,0.116338,0.165328,0.162055,0.173785,0.178097,0.116099,0.220687
3,Квартира4,0.071864,0.059284,0.111161,0.065377,0.059285,0.078457,0.232548
4,Квартира5,0.051823,0.040200,0.048225,0.039922,0.047438,0.055477,0.226591
5,Квартира6,0.072791,0.038187,0.053551,0.032315,0.051897,0.041155,0.231526


### Вывод

Результаты вычислений показали, что нужно выбрать Квартиру4 (строка № 4).